In [51]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import date
from pandas.tseries.offsets import BDay

In [52]:
token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=InstrumentsConsolidated&date={}&recaptchaToken='.format(date.today().strftime("%Y-%m-%d"))
#token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=InstrumentsConsolidated&date=2020-06-26&recaptchaToken='
r = requests.get(token)
vJson = json.loads(r.content)
vJson['token']
optionList = 'https://arquivos.b3.com.br/api/download/?token={}'.format(vJson['token'])
r1 = requests.get(optionList)

data = r1.content.decode("ISO-8859-1").splitlines()
df = pd.DataFrame(data) 
df1=df.copy()
df1.dropna(inplace = True) 
df1 = df1.applymap(str)
df1 = pd.DataFrame(df1)
df1 = df1[0].str.split(";", n = 100, expand = True)
df1.columns = df1.iloc[0]
df1 = df1[1:]
df1 = df1.reset_index()
df1.drop("index", axis = 1, inplace=True)

df2 = df1[["TckrSymb", "Asst", "SgmtNm", "SctyCtgyNm", "XprtnDt", "TradgEndDt", "OptnTp", "ExrcPric", "OptnStyle"]]
df2 = df2[df2['SctyCtgyNm']=='OPTION ON EQUITIES']
df2['ExrcPric'] = [x.replace(',', '.') for x in df2['ExrcPric']]
df2['ExrcPric'] = df2['ExrcPric'].astype(float)

In [98]:
#vCot = pd.read_excel(open(r'C:\Users\kiyo_\OneDrive\Documentos\Macro.xlsm', 'rb'), sheet_name='Sheet1') 
#vCot.drop("Asst", axis = 1, inplace=True)
#vCot.columns = ["Ticker","Value"]


In [54]:
lastBD = pd.datetime.today() - BDay(1)
lastBD = lastBD.strftime("%Y-%m-%d")
token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=TradeInformationConsolidated&date={}'.format(lastBD)
#token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=InstrumentsConsolidated&date=2020-06-26&recaptchaToken='
r = requests.get(token)
vJson = json.loads(r.content)
vJson['token']
optionList = 'https://arquivos.b3.com.br/api/download/?token={}'.format(vJson['token'])
r1 = requests.get(optionList)

data1 = r1.content.decode("ISO-8859-1").splitlines()
dfRaw2 = pd.DataFrame(data1) 
df_trade=dfRaw2.copy()
df_trade.dropna(inplace = True) 
df_trade = df_trade.applymap(str)
df_trade = pd.DataFrame(df_trade)
df_trade = df_trade[0].str.split(";", n = 100, expand = True)
df_trade.columns = df_trade.iloc[0]
df_trade = df_trade[1:]
df_trade = df_trade.reset_index()
df_trade.drop("index", axis = 1, inplace=True)
df_trade = df_trade[(df_trade['SgmtNm']=='EQUITY PUT') | (df_trade['SgmtNm']=='EQUITY CALL')]
df_trade=df_trade[(df_trade['TradQty']!='')]
df_trade["Strike"]=0.0
df_trade["Ticker"]=""
df_trade["Validade"]=""


,RptDt,TckrSymb,ISIN,SgmtNm,MinPric,MaxPric,TradAvrgPric,LastPric,AdjstdQt,RefPric,TradQty,FinInstrmQty
1533,2020-06-29,ABEVG120,BRABEV3G0712,EQUITY CALL,"2,35","2,35","2,35","2,35",,"2,29",1,4000
1534,2020-06-29,ABEVG130,BRABEV3G0753,EQUITY CALL,"1,31","1,45","1,36","1,45",,"1,39",6,23300
1535,2020-06-29,ABEVG14,BRABEV3G0787,EQUITY CALL,"0,49","0,77","0,67","0,71",,"0,67",92,145800
1536,2020-06-29,ABEVG150,BRABEV3G07H7,EQUITY CALL,"0,19","0,31","0,26","0,26",,"0,25",713,2447800
1537,2020-06-29,ABEVG158,BRABEV3G0738,EQUITY CALL,"0,11","0,14","0,11","0,13",,"0,12",5,22900
...,...,...,...,...,...,...,...,...,...,...,...,...
29857,2020-06-29,YDUQS340,BRYDUQ3S00R3,EQUITY PUT,"2,32","3,22","2,59","2,33",,"2,36",9,40800
29859,2020-06-29,YDUQS350,BRYDUQ3S00S1,EQUITY PUT,"2,88","3,83","3,55","2,88",,"2,9",6,27800
29892,2020-06-29,YDUQT300,BRYDUQ3T01Z2,EQUITY PUT,2,2,2,2,,"1,86",1,100
29894,2020-06-29,YDUQT320,BRYDUQ3T01Y5,EQUITY PUT,"2,8","3,06",3,"2,8",,"2,65",4,10000


In [105]:
df_trade["Tipo"]=""


In [106]:
for i in df_trade.TckrSymb:
    try:
        df_trade.ix[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Strike'] = df2.loc[df2["TckrSymb"]==i].ExrcPric.values[0]
        df_trade.ix[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Ticker'] = df2.loc[df2["TckrSymb"]==i].Asst.values[0]
        df_trade.ix[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Validade'] = df2.loc[df2["TckrSymb"]==i].XprtnDt.values[0]
        df_trade.ix[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Tipo'] = df2.loc[df2["TckrSymb"]==i].OptnStyle.values[0]
    except:
        print(i)

C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-

IBOVG100
IBOVG101
IBOVG103
IBOVG105
IBOVG94
IBOVG95
IBOVG96
IBOVG97
IBOVG98
IBOVG99
IBOVH100
IBOVH102
IBOVH104
IBOVH105
IBOVH107
IBOVH108
IBOVH115
IBOVH119
IBOVH92
IBOVH94
IBOVH96
IBOVJ107
IBOVJ92
IBOVJ93
IBOVL10
IBOVL104
IBOVL11
IBOVL7
IBOVS65
IBOVS70
IBOVS72
IBOVS75
IBOVS770
IBOVS80
IBOVS85
IBOVS86
IBOVS87
IBOVS88
IBOVS89
IBOVS90
IBOVS91
IBOVS92
IBOVS93
IBOVS94
IBOVS95
IBOVS96
IBOVT71
IBOVT73
IBOVT80
IBOVT81
IBOVT82
IBOVT84
IBOVT85
IBOVT87
IBOVT900
IBOVT92
IBOVT930
IBOVT96
IBOVT97
IBOVV64
IBOVV84
IBOVV85
IBOVV87
IBOVV89
IBOVV90
IBOVV94
IBOVW70
IBOVW75
IBOVX70
IBOVX79
IBOVX85
IBOVX86
IBOVX89
IBOVX970


In [84]:
#df2.loc[df2["TckrSymb"]=='ABEVG120'].FinInstrmQty.values[0]
#df2.loc[df2["TckrSymb"]=='ABEVG120'].ExrcPric.values[0]

print(df_trade.loc[df_trade["TckrSymb"]=='ABEVG120'].Strike.values[0])
print(df2.loc[df2["TckrSymb"]=='ABEVG120'].ExrcPric.values[0])



0
12.08


C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


0

In [107]:
df_trade['Ticker','Tipo','SgmtNm',]

,RptDt,TckrSymb,ISIN,SgmtNm,MinPric,MaxPric,TradAvrgPric,LastPric,AdjstdQt,RefPric,TradQty,FinInstrmQty,Strike,Ticker,Validade,Tipo
1533,2020-06-29,ABEVG120,BRABEV3G0712,EQUITY CALL,"2,35","2,35","2,35","2,35",,"2,29",1,4000,12.08,ABEV3,2020-07-20,AMER
1534,2020-06-29,ABEVG130,BRABEV3G0753,EQUITY CALL,"1,31","1,45","1,36","1,45",,"1,39",6,23300,13.08,ABEV3,2020-07-20,AMER
1535,2020-06-29,ABEVG14,BRABEV3G0787,EQUITY CALL,"0,49","0,77","0,67","0,71",,"0,67",92,145800,14.08,ABEV3,2020-07-20,AMER
1536,2020-06-29,ABEVG150,BRABEV3G07H7,EQUITY CALL,"0,19","0,31","0,26","0,26",,"0,25",713,2447800,15.08,ABEV3,2020-07-20,AMER
1537,2020-06-29,ABEVG158,BRABEV3G0738,EQUITY CALL,"0,11","0,14","0,11","0,13",,"0,12",5,22900,15.83,ABEV3,2020-07-20,EURO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29857,2020-06-29,YDUQS340,BRYDUQ3S00R3,EQUITY PUT,"2,32","3,22","2,59","2,33",,"2,36",9,40800,34.00,YDUQ3,2020-07-20,EURO
29859,2020-06-29,YDUQS350,BRYDUQ3S00S1,EQUITY PUT,"2,88","3,83","3,55","2,88",,"2,9",6,27800,35.00,YDUQ3,2020-07-20,EURO
29892,2020-06-29,YDUQT300,BRYDUQ3T01Z2,EQUITY PUT,2,2,2,2,,"1,86",1,100,30.00,YDUQ3,2020-08-17,EURO
29894,2020-06-29,YDUQT320,BRYDUQ3T01Y5,EQUITY PUT,"2,8","3,06",3,"2,8",,"2,65",4,10000,32.00,YDUQ3,2020-08-17,EURO


In [126]:
df_List = df_trade[['Ticker','Tipo','SgmtNm','TckrSymb','Validade','Strike','TradQty']]

In [127]:
df_List=df_List[df_List["Ticker"]!=""]

In [129]:
df_List.to_csv("teste1.csv")

,Ticker,Tipo,SgmtNm,TckrSymb,Validade,Strike,TradQty
1533,ABEV3,AMER,EQUITY CALL,ABEVG120,2020-07-20,12.08,1
1534,ABEV3,AMER,EQUITY CALL,ABEVG130,2020-07-20,13.08,6
1535,ABEV3,AMER,EQUITY CALL,ABEVG14,2020-07-20,14.08,92
1536,ABEV3,AMER,EQUITY CALL,ABEVG150,2020-07-20,15.08,713
1537,ABEV3,EURO,EQUITY CALL,ABEVG158,2020-07-20,15.83,5
...,...,...,...,...,...,...,...
29857,YDUQ3,EURO,EQUITY PUT,YDUQS340,2020-07-20,34.00,9
29859,YDUQ3,EURO,EQUITY PUT,YDUQS350,2020-07-20,35.00,6
29892,YDUQ3,EURO,EQUITY PUT,YDUQT300,2020-08-17,30.00,1
29894,YDUQ3,EURO,EQUITY PUT,YDUQT320,2020-08-17,32.00,4
